<a href="https://colab.research.google.com/github/tanjkdarryl/Intermediate_26/blob/main/Audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pip install packages

In [ ]:
!pip install torch torchvision torchaudio
!pip install librosa streamlit pandas scikit-learn
!pip install soundfile

  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (24.6 MB)


Packages

In [ ]:
# Install Streamlit related packages
!pip install streamlit
!pip install streamlit streamlit_option_menu
!pip install -q streamlit

# Install localtunnel for exposing the app
!npm install localtunnel

# Create the 'pages' directory where your multi-page app files will reside
!mkdir -p pages

⠙⠹⠸⠼⠴⠦⠧
up to date, audited 23 packages in 1s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠧

Home.py

In [ ]:
%%writefile Home.py
import streamlit as st

# Set Streamlit page configuration
# page_title sets the browser tab title AND the label for this page in the sidebar
st.set_page_config(
    page_title="Home", # This will be the label for the Home.py page in the sidebar
    layout="centered", # Sets the page layout to centered
    initial_sidebar_state="expanded", # Keeps the sidebar expanded by default
)

# --- Content for the Home Page ---
st.title("Welcome to the Home Page! 🏠")
st.write("This is the main entry point of our Streamlit application.")
st.markdown("""
    Hello there! You've landed on the **Home Page**.
    Use the sidebar on the left to navigate to other sections.
""")

st.info("To see the navigation, click the `>` icon in the top-left corner of the page (if the sidebar is hidden).")

Overwriting Home.py


In [ ]:
%%writefile pages/1_About.py
import streamlit as st

# Set page config for consistency across pages (optional, but good practice)
st.set_page_config(layout="centered")

st.title("About Tommo: Your Digital Companion 💡")

st.markdown("""
    At [Your Hackathon Team Name - e.g., Team A.I. for Good], we envision a future where emotional well-being is accessible and proactively supported for everyone.
    Introducing **Tommo**, your innovative digital companion designed to elevate mental health support, especially for the disadvantaged and disabled in our society.
""")

st.subheader("What is Tommo?")
st.markdown("""
    Tommo, named after the Japanese word for friend ('Tomodachi'), embodies true companionship.
    It's more than just an application; it's a reliable touchpoint that helps individuals document their journey towards better mental well-being.
    Our solution acts not just as a bridge between an individual and happiness, but also as a secure space to record feelings and expressions, serving as an impetus to better mental health.
""")

st.subheader("How Tommo Empowers & Assists")
st.markdown("""
    Tommo leverages cutting-edge AI, including sophisticated **facial and speech recognition** capabilities, to understand and interact with users empathetically.
    At its core, a powerful **LLM (Large Language Model) powered by PyTorch** ensures secure, private, and helpful conversations, fostering a genuine sense of connection.

    ### Reducing Therapists' Initial Efforts
    A core aspect of Tommo's innovation is its ability to support mental health professionals. By consistently recording an individual's emotional states, expressions, and conversational insights over time, Tommo provides therapists with invaluable, objective data. This rich context is available *before* the first session, allowing therapists to:
    * **Understand the user's emotional baseline and fluctuations.**
    * **Identify recurring patterns or triggers.**
    * **Tailor initial interventions more effectively.**
    This significantly reduces the initial investigative effort for therapists, enabling them to focus on targeted, impactful therapeutic work from the outset, leading to more efficient and personalized care.
""")

st.subheader("Our Vision: Empowering Through AI")
st.markdown("""
    Tommo directly addresses the guiding question of the Hackathon:
    _"How may we improve accessibility and empower the disadvantaged and disabled in our society?"_
    By providing an accessible, non-judgmental, and technologically advanced platform, Tommo empowers individuals to proactively engage with their emotional well-being, breaking down barriers to mental health support.
""")

Overwriting pages/1_About.py


Pulling RAVDESS dataset through Kaggle.json via API

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (3).json


{'kaggle (3).json': b'{"username":"oneciuswang","key":"1eb25e13a1dc7ecb37900424f1859ad5"}'}

Move token to the correct location and set permissions

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Downloading RAVDESS dataset from Kaggle (Training)

---



---



---



In [ ]:
!pip install -q kaggle

# List available datasets if unsure
!kaggle datasets list -s ravdess

# Download the dataset
!kaggle datasets download -d uwrfkaggler/ravdess-emotional-speech-audio

# Unzip it
!unzip -q ravdess-emotional-speech-audio.zip -d ravdess

In [ ]:
import os

DATA_DIR = "./ravdess"
file_paths = [os.path.join(DATA_DIR, f) for f in os.listdir(DATA_DIR) if f.endswith(".wav")]


Preprocess Audio Data for Emotion Labels

1.   List item
2.   List item



In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm

# Define path to the unzipped dataset
DATASET_PATH = "ravdess"

# Define a function to extract emotion from filename
def extract_emotion(filename):
    # Example: 03-01-05-01-01-01-12.wav → emotion label is the 3rd value
    emotion_map = {
        '01': 'neutral',
        '02': 'calm',
        '03': 'happy',
        '04': 'sad',
        '05': 'angry',
        '06': 'fearful',
        '07': 'disgust',
        '08': 'surprised'
    }
    parts = filename.split("-")
    return emotion_map.get(parts[2], "unknown")

# Extract features from all audio files
def extract_features(dataset_path):
    features = []
    for root, _, files in os.walk(dataset_path):
      for file in files:
        if file.endswith(".wav"):
            emotion = extract_emotion(file)
            filepath = os.path.join(root, file)
            y, sr = librosa.load(filepath, duration=3, offset=0.5)
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
            mfcc_scaled = np.mean(mfcc.T, axis=0)
            features.append([mfcc_scaled, emotion])
    return pd.DataFrame(features, columns=['features', 'label'])

# Process data
data_df = extract_features(DATASET_PATH)
#Check data
print("Number of samples extracted:", len(data_df))
print(data_df['label'].value_counts())

# Encode labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data_df['encoded_label'] = le.fit_transform(data_df['label'])

# Final data
X = np.array(data_df['features'].tolist())
y = np.array(data_df['encoded_label'])

Number of samples extracted: 2880
label
calm         384
fearful      384
disgust      384
happy        384
surprised    384
sad          384
angry        384
neutral      192
Name: count, dtype: int64


Train a PyTorch Classifier for emotion classification


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# Convert to tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# Train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Define a simple neural network
class EmotionClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(40, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, len(le.classes_))  # number of classes
        )

    def forward(self, x):
        return self.model(x)

model = EmotionClassifier()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(15):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = loss_fn(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 290.0570
Epoch 2, Loss: 170.0601
Epoch 3, Loss: 152.2772
Epoch 4, Loss: 143.2470
Epoch 5, Loss: 137.2748
Epoch 6, Loss: 131.9758
Epoch 7, Loss: 127.5243
Epoch 8, Loss: 125.2670
Epoch 9, Loss: 123.0159
Epoch 10, Loss: 119.1990
Epoch 11, Loss: 117.5532
Epoch 12, Loss: 114.8345
Epoch 13, Loss: 112.2605
Epoch 14, Loss: 112.2023
Epoch 15, Loss: 110.0174


SImple UI for users to upload .wav audio files

In [ ]:
%%writefile pages/1_chatbot.py
import streamlit as st
import librosa
import numpy as np
import torch
import torch.nn as nn

# Load model definition
class EmotionClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(40, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 8)  # 8 emotion classes
        )

    def forward(self, x):
        return self.model(x)

# Load trained model
model = EmotionClassifier()
model.load_state_dict(torch.load('emotion_model.pth', map_location=torch.device('cpu')))
model.eval()

# Label decoder (update to match your model)
label_map = ['angry', 'calm', 'disgust', 'fearful', 'happy', 'neutral', 'sad', 'surprised']

# UI
st.title("Speech Emotion Detection for Depression Risk Screening")
st.write("Upload a `.wav` file and we'll analyze emotional tone.")

uploaded_file = st.file_uploader("Choose a .wav file", type="wav")

if uploaded_file is not None:
    y, sr = librosa.load(uploaded_file, duration=3, offset=0.5)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    mfcc_scaled = np.mean(mfcc.T, axis=0)
    input_tensor = torch.tensor(mfcc_scaled, dtype=torch.float32).unsqueeze(0)

    with torch.no_grad():
        output = model(input_tensor)
        pred_idx = torch.argmax(output).item()
        emotion = label_map[pred_idx]

    st.success(f"Detected Emotion: **{emotion}**")
    if emotion in ['sad', 'fearful', 'angry']:
        st.warning("⚠️ Possible signs of negative emotional tone. Consider further evaluation.")
    else:
        st.info("🙂 No immediate signs of depressive tone.")

Overwriting pages/1_chatbot.py


Run Streamlit app

In [ ]:
# Save model
torch.save(model.state_dict(), 'emotion_model.pth')

#Run streamlit
!streamlit run Home.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.229.131.252
⠙⠹your url is: https://cruel-seals-dance.loca.lt
